In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

# Northland Geology and Roads

This Jupyter Notebook is intended to help you visualize the anomalies of interest against the Geologic Map and the Topographic Map with roads.  It is also a good example on how to retrieve information from [Macrostrat.org](https://macrostrat.org/map/#/z=9.0/x=174.1666/y=-35.5429/bedrock/lines/) for New Zealand.  All maps are in projection EPGS:3857.

## SplitMap with ImageOverlay

This example shows how to use the [Leaflet SplitMap](https://github.com/QuantStack/leaflet-splitmap) slider with ipyleaflet.  A raster is added as an image overlay.  The opacity of the raster can be changed before running the cells of the notebook.   

Details of units in the geologic map can be found here [Macrostrat.org](https://macrostrat.org/map/#/z=9.0/x=174.1666/y=-35.5429/bedrock/lines/).  Macrostrat data can be browsed through the [Rockd.org](https://rockd.org/) mobile app, which is a good resource for geologists and geoscientists alike. 

## Check the metadata on the geologic unit

To check the metadata of a particular unit, click on it and press the "Check Formation!" button.


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import io
import os
import requests
from IPython.display import display,clear_output
import ipywidgets as widgets
from ipywidgets import Label,Dropdown,jslink
from ipyleaflet import (
    Map, TileLayer, SplitMapControl, ScaleControl,FullScreenControl,ImageOverlay,LayersControl,WidgetControl,Marker,
    DrawControl,MeasureControl
)

In [3]:
center = [-35.611896, 174.185050] #-34.8252978589571, 173.54580993652344
zoom = 10
opacity=0.8

In [4]:
m = Map(center=center, zoom=zoom)

left = TileLayer(url='https://tiles.macrostrat.org/carto/{z}/{x}/{y}.png')
right = TileLayer(url='https://{s}.tile.opentopomap.org/{z}/{x}/{y}.png')

#Image Overlay for a grid
#image = ImageOverlay(
#    url='RTP.png',
#    # url='../06Q1fSz.png',
#    bounds=((-35.611896, 174.185050), (-35.411021, 174.380248)),
#    opacity=opacity
#)

#image = ImageOverlay(
#    url='Northland_CB_2.png',
#    bounds=((-37.0078735, 171.9919065), (-34.0078135, 174.9919665)),
#    opacity=opacity
#)

#image = ImageOverlay(
#    url='Northland_TMI_reprojected.png',
#    # url='../06Q1fSz.png',
#    bounds=((-36.4081355,172.6260946), (-34.3687166,174.6581728)),
#    opacity=opacity
#)

image = ImageOverlay(
    url='Northland_TMI_RTP_reprojected.png',
    bounds=((-36.4081355,172.6260946), (-34.3687166,174.6581728)),
    opacity=opacity
)

#Add the layer
m.add_layer(image)
control = LayersControl(position='topright')
m.add_control(control)
m

#Split the map
control = SplitMapControl(left_layer=left, right_layer=right)
m.add_control(control)
m.add_control(ScaleControl(position='bottomleft',max_width=200))
m.add_control(FullScreenControl())
#m

#Display coordinates on click
def handle_interaction(**kwargs):
    if kwargs.get('type') == 'mousedown': #'mousemove' will update coordinates with every position
        label.value = str(kwargs.get('coordinates'))
        
label = Label()
display(label)
m.on_interaction(handle_interaction)

#Button to check the formation on the geographic location selected
button = widgets.Button(description="Check Formation!")
output = widgets.Output()

display(button, output)

def on_button_clicked(b):
    with output:
        if label.value=='':
            clear_output(wait=True)
            print('Click on a geologic formation on the map and click to get the info')
        else:
            #Get the infor from the API
            clear_output(wait=True)
            lat = label.value.strip('][').split(', ')[0] #Break the label into usable strings
            lon = label.value.strip('][').split(', ')[1] #Break the label into usable strings
            url='https://macrostrat.org/api/v2/geologic_units/map?lat='+lat+'&lng='+lon+'&scale=large&response=short&format=csv'
            s=requests.get(url).content
            c=pd.read_csv(io.StringIO(s.decode('utf-8')))
            #Make it into a Pandas DataFrame
            df=pd.DataFrame(c)
            display(df)

button.on_click(on_button_clicked)

#Show the maps  
m

Label(value='')

Button(description='Check Formation!', style=ButtonStyle())

Output()

Map(center=[-35.611896, 174.18505], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title'…

# Retrieve more information from the API on Geological Units

We can request the geojson file for a geologic unit given the geographic coordinates.  The url should be formatted as in the example code below.

In [5]:
from ipyleaflet import Map, ImageOverlay,GeoJSON
m = Map(center=(-35.611896, 174.185050), zoom=7)

#Request info from the API 
if label.value=='':
    print('Click on a geologic formation on the map above and run this cell to get the info and polygon')
else:
    lat = label.value.strip('][').split(', ')[0] #Break the label into usable strings
    lon = label.value.strip('][').split(', ')[1] #Break the label into usable strings
    #Request the geojson file from the API    
    import urllib.request, json 
    with urllib.request.urlopen('https://macrostrat.org/api/v2/geologic_units/map?lat='+lat+'&lng='+lon+'&response=short&scale=large&format=geojson_bare') as url:
        data = json.loads(url.read().decode())
    #Get the infor from the API
    url='https://macrostrat.org/api/v2/geologic_units/map?lat='+lat+'&lng='+lon+'&scale=large&response=short&format=csv'
    s=requests.get(url).content
    c=pd.read_csv(io.StringIO(s.decode('utf-8')))
    #Make it into a Pandas DataFrame
    df=pd.DataFrame(c)
    display(df)

    geo_json = GeoJSON(
        data=data,
        style={'color': 'red','opacity': 1,'fillOpacity': 0.7} #If Color from the API change 'color' to df.color[0]
    )
    m.add_layer(geo_json)
#Add draw control
draw_control = DrawControl()
draw_control.polyline =  {
    "shapeOptions": {
        "color": "#6bc2e5",
        "weight": 8,
        "opacity": 1.0
    }
}
draw_control.polygon = {
    "shapeOptions": {
        "fillColor": "#6be5c3",
        "color": "#6be5c3",
        "fillOpacity": 1.0
    },
    "drawError": {
        "color": "#dd253b",
        "message": "Oups!"
    },
    "allowIntersection": False
}
draw_control.circle = {
    "shapeOptions": {
        "fillColor": "#efed69",
        "color": "#efed69",
        "fillOpacity": 1.0
    }
}
draw_control.rectangle = {
    "shapeOptions": {
        "fillColor": "#fca45d",
        "color": "#fca45d",
        "fillOpacity": 1.0
    }
}

m.add_control(draw_control)

#Measure Control
measure = MeasureControl(
    position='bottomleft',
    active_color = 'orange',
    primary_length_unit = 'kilometers'
)
m.add_control(measure)

measure.completed_color = 'red'

measure.add_length_unit('yards', 1.09361, 4)
measure.secondary_length_unit = 'yards'

measure.add_area_unit('sqyards', 1.19599, 4)
measure.secondary_area_unit = 'sqyards'

m

Click on a geologic formation on the map above and run this cell to get the info and polygon


Map(center=[-35.611896, 174.18505], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title'…

# Northland EQs

The EQs of Northland displayed using the Circle marker and Popup tool of ipyleaflet. 

In [6]:
from ipywidgets import HTML
from ipyleaflet import Circle,Popup
import numpy as np
import matplotlib.pyplot as plt
#Load the data for EQs
long,lati,depth,mag=np.loadtxt('EQs_Northland.dat',skiprows=1,unpack=True)

#Create the map
m2 = Map(center=(-35.611896, 174.185050), zoom=7)

circles = []
for i in range(0,len(long)):
    c = Circle(location=(lati[i],long[i]), radius=np.int(10*depth[i]),name=np.str(depth[i]))
    circles.append(c)
    m2.add_layer(c)
    
#Display values with a popup
def handle_interaction2(**kwargs):
    if kwargs.get('type') == 'mousedown': #'mousemove' will update coordinates with every position
        label.value = str(kwargs.get('coordinates'))
        lt = float(label.value.strip('][').split(', ')[0]) #Break the label into usable strings
        lng = float(label.value.strip('][').split(', ')[1]) #Break the label into usable strings
        message=HTML()
        p=np.where((long==lng)&(lati==lt)) #Find the locations  to extract the value
        if len(p[0])>1:
            message.value = np.str(depth[p].mean())+' km'+' <b>Mw:</b> '+np.str(mag[p].mean())
        elif len(p[0])==1:
            message.value = np.str(depth[p])+' km'+' <b>Mw:</b> '+np.str(mag[p])
        else:
            message.value = 'No data'
        # Popup with a given location on the map:
        popup = Popup(
        location=(lt,lng),
        child=message,
        close_button=False,
        auto_close=False,
        close_on_escape_key=False
        )
        m2.add_layer(popup)

label2 = Label()
display(label2)
m2.on_interaction(handle_interaction2)
m2

Label(value='')

Map(center=[-35.611896, 174.18505], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title'…

# Epithermal vector data over Northland

the data available can be obtained from the [New Zealand Petroleum and Minerals](https://www.nzpam.govt.nz/) database (datapack MR4343).  The shapefiles were reprojected into EPSG:4326 as GeoJSON files for visualization on ipyleaflet from NZMG (EPSG:27200) coordinates. 

In [56]:
from ipyleaflet import Map, ImageOverlay,GeoJSON,json
center = (-35.5, 174.6) #or your desired coordinates
m = Map(center=center, zoom=6)
#m = Map(center=center, zoom=6)

#left = TileLayer(url='https://tiles.macrostrat.org/carto/{z}/{x}/{y}.png')
#right = TileLayer(url='https://{s}.tile.opentopomap.org/{z}/{x}/{y}.png')

with open('./Vector_data_GeoJSON/hydrothm.geojson','r') as f: #epiareas.geojson
    data = json.load(f)
    geo_json = GeoJSON(data=data, style = {'color': 'Blue', 'opacity':1, 'weight':1.9, 'dashArray':'9', 'fillOpacity':0.1})
m.add_layer(geo_json)

#Split the map
#control = SplitMapControl(left_layer=left, right_layer=right)
#m.add_control(control)
m.add_control(ScaleControl(position='bottomleft',max_width=200))
m.add_control(FullScreenControl())
m

Map(center=[-35.5, 174.6], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_o…